In [1]:
import sys
sys.path.append('../../src/')

In [27]:
%load_ext autoreload
%autoreload 2

from data import load_data
from multilabel.loader import MultiLabelDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [21]:
data_dir = "../../data/xray8"
images_dir = "../../data/xray8/"

In [22]:
train = pd.read_csv(os.path.join(data_dir,"train.csv"))
val = pd.read_csv(os.path.join(data_dir,"val.csv"))
test = pd.read_csv(os.path.join(data_dir,"test.csv"))

In [23]:
train.head(2)

,Image Index,Patient ID,Effusion,No Finding,Mass,Nodule,Atelectasis,Pneumothorax,Consolidation,FilePath
0,00017638_000.png,17638,0,1,0,0,0,0,0,images_008/images/00017638_000.png
1,00011258_000.png,11258,0,1,0,0,0,0,0,images_005/images/00011258_000.png


In [24]:
num_workers = 8
batch_size = 64
image_size=299

In [29]:
aug = T.Compose([T.Resize(size=image_size),
                 T.ToTensor(),
                 T.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1))])


train = pd.read_csv(os.path.join(data_dir, "train.csv")).drop(['FilePath', 'Patient ID'], axis=1).set_index('Image Index')
test = pd.read_csv(os.path.join(data_dir, "test.csv")).drop(['FilePath', 'Patient ID'], axis=1).set_index('Image Index')
val = pd.read_csv(os.path.join(data_dir, "val.csv")).drop(['FilePath', 'Patient ID'], axis=1).set_index('Image Index')

train_dataset = MultiLabelDataset(root=os.path.join(data_dir, "train"),
                                     dataframe=train,
                                     transform=aug)
val_dataset = MultiLabelDataset(root=os.path.join(data_dir, "val"),
                                   dataframe=val,
                                   transform=aug)
test_dataset = MultiLabelDataset(root=os.path.join(data_dir, "test"),
                                    dataframe=test,
                                    transform=aug)

data_loader_train = td.DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=num_workers,
                                  pin_memory=True)
data_loader_val = td.DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                drop_last=False,
                                num_workers=num_workers,
                                pin_memory=True)
data_loader_test = td.DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 drop_last=False,
                                 num_workers=num_workers,
                                 pin_memory=True)

In [30]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_train):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|████████████████████████████████████████████████████████████████████████████████| 320/320 [00:38<00:00,  8.30it/s]


In [31]:
count = len(train) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4955, 0.4955, 0.4955])
std:  tensor([0.2892, 0.2892, 0.2892])


In [32]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_val):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:15<00:00,  4.20it/s]


In [33]:
count = len(val) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4956, 0.4956, 0.4956])
std:  tensor([0.2889, 0.2889, 0.2889])


In [34]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_test):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [00:15<00:00,  4.27it/s]


In [35]:
count = len(test) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.4957, 0.4957, 0.4957])
std:  tensor([0.2889, 0.2889, 0.2889])


## Calculating pos_weights

In [37]:
def calculate_pos_weights(class_counts,data):
    pos_weights = np.ones_like(class_counts)
    neg_counts = [len(data)-pos_count for pos_count in class_counts]
    for cdx, (pos_count, neg_count) in enumerate(zip(class_counts,  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5)

    return torch.as_tensor(pos_weights, dtype=torch.float)

In [49]:
calculate_pos_weights([5158, 4896, 2771, 2857, 4974, 2502, 2028], data_loader_train.dataset)

tensor([2., 3., 6., 6., 3., 7., 9.])

In [53]:
import torch
import torch.nn as nn

In [57]:
a = None
b = None
for inp, tar in data_loader_train:
    a = inp
    b = tar
    break

In [ ]:
[2., 3., 6., 6., 3., 7., 9.]

In [64]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(torch.zeros(64, 7), b)
print(loss)

criterion_weighted = nn.BCEWithLogitsLoss(pos_weight=torch.as_tensor([2., 3., 6., 6., 3., 7., 9.], dtype=torch.float))
loss_weighted = criterion_weighted(torch.zeros(64, 7), b)
print(loss_weighted)

tensor(0.6931)
tensor(0.8912)


In [69]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(torch.zeros(64, 7), b)
print(loss)

criterion_weighted = nn.BCEWithLogitsLoss(pos_weight=torch.as_tensor(20450*7/25186, dtype=torch.float))
loss_weighted = criterion_weighted(torch.zeros(64, 7), b)
print(loss_weighted)

tensor(0.6931)
tensor(1.1569)
